<a href="https://colab.research.google.com/github/tanthongtan/ptm/blob/master/sam_used_in_experiments_unbiased_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hyperparameters

In [ ]:
num_topic = 50
dataset = 'nytimes'
method = 'sam'

#model hyperparameters
alpha_scalar = 0.5
c0 = 1000.0
kappa1 = 2500.0
prior_mu = 'neg' #possible options: neg, pos, mean

#GMC hyperparameters
num_samples = 1
num_burn = 50000
S = 68000
L = 2
decay_factor = 1/3
eta_theta = 1e-1
rho_theta = 1e-1
eta_mu = 2.5e-4
rho_mu = 1e-1

# Run GMC Inference

In [ ]:
#only for google colab
import sys
import os
if 'google.colab' in sys.modules:
    #lets see what gpu we were given
    !nvidia-smi
    #get repository
    !git clone https://github.com/tanthongtan/ptm.git
    %cd '/content/ptm'
    #get ref corp if doesn't exist
    if not os.path.isdir('wiki_final'):
        !unzip -q "/content/drive/My Drive/wiki_final.zip"

import torch
import torch.nn.functional as F
from geodesic import GeodesicMonteCarlo
from dataset import load_data, csr_to_torchsparse
import geodesic as g
import distributions as D
from tqdm.notebook import tqdm
import torch.distributions as dist
import numpy as np
import time
from utils import print_topics, get_topics, vmf_perplexity, clustering_metrics_20news, print_summary, get_invalid_topics

#make all tensors cuda if available and double
if torch.cuda.is_available():
    torch.set_default_tensor_type(torch.cuda.DoubleTensor)
    gpu = True
else:
    torch.set_default_tensor_type(torch.DoubleTensor)
    gpu = False

#Load Data
data_tr, data_te, vocab, vocab_size, num_tr = load_data(use_tfidf = True, sublinear = False, normalize = True, dataset = dataset)
tensor_te = csr_to_torchsparse(data_te, gpu)
tensor_tr = csr_to_torchsparse(data_tr, gpu)


Fri Sep 29 03:20:08 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    45W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%capture cap


#declare tensor hyperparameters
alpha = torch.full((1,num_topic), alpha_scalar)
if prior_mu == 'neg':
    mu0 = F.normalize(torch.full((vocab_size,),-1.0),dim=-1)
if prior_mu == 'pos':
    mu0 = F.normalize(torch.full((vocab_size,),1.0),dim=-1)
if prior_mu == 'mean':
    mu0 = F.normalize(torch.sparse.sum(tensor_tr,dim=0).to_dense(),dim=-1)

#randomly initialize model parameters
theta = torch.randn(num_tr,num_topic-1)
mu = F.normalize(torch.randn(num_topic, vocab_size) / (vocab_size ** 0.5) + mu0, p=2, dim=-1)

#declare GMC transition kernels
kernel = GeodesicMonteCarlo(L)
params = {'theta': theta, 'mu':mu}
init_etas = {'theta': eta_theta, 'mu':eta_mu}
geodesics = {'theta': g.RnGeodesic(eta = eta_theta, rho = rho_theta), 'mu': g.SphericalGeodesic(eta = eta_mu, rho = rho_mu)}
vs = {name: geodesics[name].projection(params[name],dist.MultivariateNormal(torch.zeros(params[name].shape[-1]), torch.eye(params[name].shape[-1])).sample([params[name].shape[0]])) for name in params}

#start sampling loop
t = tqdm(range(num_samples+num_burn))
theta_samples = 0
mu_samples = 0
mu_save_collection_samples = [] #this is to test for coherence by time
start_time = time.time() #get start time
sampling_its = {50, 100, 200, 400, 800, 1600, 3200, 6400, 12800, 25600, 51200, 102400, num_burn}
#basically get torchsparse and check if stochastic gradient or not
idx = torch.randperm(num_tr)[:S]
x_batch = csr_to_torchsparse(data_tr[idx.cpu()], gpu)
stochastic_gradient = False
if idx.shape[0] < num_tr:
    stochastic_gradient = True
for i in t:

    #we only need to draw random sample if stochastic gradient, save time for full gradient
    if stochastic_gradient:
        idx = torch.randperm(num_tr)[:S]
        x_batch = csr_to_torchsparse(data_tr[idx.cpu()], gpu)

    for name in geodesics:
        geodesics[name].eta = init_etas[name] * ((int(i/L)+1) ** (-(decay_factor)))
    params, vs = kernel.stochastic_transition(params, vs, geodesics, D.SamJointDistributionWithStickDirUnbiased(x_batch, alpha, c0, mu0, kappa1, idx))

    theta = params['theta']
    mu = params['mu']

    if torch.any(mu != mu):
        break

    if i >= num_burn:
        theta_samples += theta
        mu_samples += mu

    if i % (int(num_burn/50)) == 0 or i in sampling_its:
        print("\ncurrent iteration:", i)
        print("elapsed time", time.time() - start_time)
        print("mu norms", mu.norm(dim=-1).sum(), num_topic)
        print("sparsity",(torch.abs(mu)**2.).norm(dim=-1))
        print("sparsitymean",(torch.abs(mu)**2.).norm(dim=-1).mean())
        pi = dist.StickBreakingTransform()(theta)
        print("pi sums", pi.sum(dim=-1).sum(), num_tr)

        sum_ll = 0.0
        sum_cs = 0.0
        for j in range(int(np.ceil(num_tr/S))):
            curr_pi = pi[j*S:j*S+S]
            curr_tensor_tr = csr_to_torchsparse(data_tr[j*S:j*S+S], gpu)
            curr_avg = F.normalize(torch.matmul(curr_pi,mu), p=2, dim=-1)
            sum_ll += D.log_prob_von_mises_fisher(kappa1 * curr_avg, curr_tensor_tr).sum()
            sum_cs += D.sparse_dense_dot(curr_tensor_tr, curr_avg).sum()

        print("log likelihood", sum_ll / num_tr)
        print("cosine similarity", sum_cs / num_tr)
        print("perplexity", vmf_perplexity(tensor_te, mu, kappa1, alpha, N=1000),"\n")

        sum_cs_spread = 0
        count_cs = 0
        for j in range(num_topic-1):
            for k in range(j+1,num_topic):
                sum_cs_spread += (mu[j] * mu[k]).sum(dim=-1)
                count_cs += 1
        print("mean cs spread", sum_cs_spread / count_cs,"\n")

        pi_cpu = pi.cpu().numpy()
        print("invalid topics")
        print("normal thres", get_invalid_topics(pi_cpu,np.array(1)))
        print("thres 2x", get_invalid_topics(pi_cpu, np.array(1), 1/(num_topic*2)))
        print("thres 10x", get_invalid_topics(pi_cpu, np.array(1), 1/(num_topic*10)))
        print()

    if i % int(num_burn/5) == 0:
        emb = mu.cpu().numpy()
        print_topics(get_topics(emb,vocab))
        print("")

    if i in sampling_its:
        mu_save_collection_samples.append(mu.cpu().numpy())

#get topic coherence
mu_final = mu_samples / num_samples
theta_final = theta_samples / num_samples
print('\nalpha = ', alpha_scalar)
print("final perplexity", vmf_perplexity(tensor_te, mu_final, kappa1, alpha, N=1000))
print('prior_mu:', prior_mu)
print('kappa1:', kappa1)
print('c0:', c0)
print('decay_factor:', decay_factor)
emb = mu_final.cpu().numpy()
topics = get_topics(emb, vocab)
print_summary(topics,method,dataset)

if dataset == '20news':
    pi = dist.StickBreakingTransform()(theta_final)
    pi = pi.cpu().numpy()
    clustering_metrics_20news(pi)

In [ ]:
import random
run = random.randint(0,100000)
current_run_name = "method="+str(method)+", alpha="+str(alpha_scalar)+", K="+str(num_topic)+", dataset="+str(dataset)+ ", prior_mu="+str(prior_mu)+ ", kappa1="+str(kappa1)+ ", c0="+str(c0) + ", decay_factor="+str(round(decay_factor,3))+ ", L="+str(L) + ", num_burn="+str(num_burn) + ", run="+ str(run)
dir_first_part = str(kappa1) + "/" + str(num_topic) + "/" + str(alpha_scalar) + "/" + str(dataset) + "/"
dirname = '/content/drive/My Drive/masters_results_unbiased_double_real/sam/'+ dir_first_part + current_run_name + "/"
os.makedirs(dirname, exist_ok=False)
filename = current_run_name +".txt"
with open(dirname + filename, 'w') as f:
     f.write(cap.stdout)

#save mus
emb_filename = current_run_name +" emb.npy"

with open(dirname +emb_filename, 'wb') as f:
    np.save(f, emb)

#save means
mean_filename = current_run_name +" mean.npy"

with open(dirname + mean_filename, 'wb') as f:
    np.save(f, mu0.cpu().numpy())

#save pis
pi = dist.StickBreakingTransform()(theta_final)
pi = pi.cpu().numpy()

pi_filename = current_run_name + " pi.npy"

with open(dirname + pi_filename, 'wb') as f:
    np.save(f, pi)

#save sample collections
mu_collection_filename = current_run_name + " mucollection.npz"
with open(dirname + mu_collection_filename, 'wb') as f:
    np.savez_compressed(f, *mu_save_collection_samples)

In [ ]:
from google.colab import runtime
runtime.unassign()